## Importing all the dependencies

In [7]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
import time
import random


## Scraping

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.Realtor_db
collection = db.Homes_For_Sale_CA

In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

#URL and browser


url = 'https://www.realtor.com/realestateandhomes-search/San-Francisco-County_CA'
browser.visit(url)


html = browser.html
soup = BeautifulSoup(html, 'html.parser')
property_type=soup.find_all('span', class_='jsx-2810661766')
time.sleep(10)
price=soup.find_all('span',class_="sc-ptDSg eYGQDi")
time.sleep(10)
prop_details=soup.find_all('ul',class_='jsx-946479843 property-meta list-unstyled property-meta-srpPage')
time.sleep(10)
address=soup.find_all('div',class_='jsx-2810661766 address ellipsis srp-page-address')
time.sleep(10)
link=soup.find_all('div',class_="jsx-2810661766 detail-wrap has-cta fixed-wrapper")
time.sleep(10)
    
    
print(len(property_type))
print(len(price))
print(len(prop_details))
print(len(address))
print(len(link))

In [ ]:
prop_details[4]

In [ ]:
prop_details_dict={}

for i in range(0,len(prop_details)):
    label_list=prop_details[i].find_all("span",class_="jsx-946479843 meta-label")
    value_list=prop_details[i].find_all("span",class_="jsx-946479843 meta-value")
    for i in range(0,len(label_list)):
        if(label_list[i].text=="bed"):
            prop_details_dict["Bed"]=value_list[i].text
        elif(label_list[i].text=="bath"):
            prop_details_dict["Bath"]=value_list[i].text
        elif(label_list[i].text=="sqft"):
            prop_details_dict["House Square Feet"]=int(''.join(n for n in value_list[i].text if n.isdigit()))
        elif(label_list[i].text=="sqft lot" or label_list[i].text=="acre lot" ):
            prop_details_dict["Lot Square Feet"]=int(''.join(n for n in value_list[i].text if n.isdigit()))
        else:
            prop_details_dict[label_list[i].text]=value_list[i].text

    print(prop_details_dict) 

    #converting price to integer
    price[i]=int(''.join(n for n in price[i].text if n.isdigit()))
    print(price[i])

    #finding Zipcode from address
    zip_code=''
    for j in range(1,6):
        zip_code=zip_code+address[i].text[-j]
        zip_code=zip_code[::-1]
        print(zip_code)
    
    #link
    link[i]=f'https://www.realtor.com/{link[i].a["href"]}'
    print(link[i])
    dictionary to post in mongo DB
    post={
        "property_type":property_type[i].text,
        "Price":price[i],
        "No of beds":prop_details_dict['Bed'],
        "No of bath":prop_details_dict['Bath'],
        "Sqaure Feet":prop_details_dict['House Square Feet'],
        "Lot Square Feet":prop_details_dict['Lot Square Feet'],
        "Reference Link":link[i],
        "Address":address[i].text,
        "Zip-Code":zip_code
    }

    collection.insert_one(post)
    print(insert_success)
    

In [ ]:
browser.quit

In [3]:
url_list=[]
url_list.append('https://www.realtor.com/realestateandhomes-search/California')
for i in range(2,207):
    url=f'https://www.realtor.com/realestateandhomes-search/California/pg-{i}'
    url_list.append(url)
url_list
    

['https://www.realtor.com/realestateandhomes-search/California',
 'https://www.realtor.com/realestateandhomes-search/California/pg-2',
 'https://www.realtor.com/realestateandhomes-search/California/pg-3',
 'https://www.realtor.com/realestateandhomes-search/California/pg-4',
 'https://www.realtor.com/realestateandhomes-search/California/pg-5',
 'https://www.realtor.com/realestateandhomes-search/California/pg-6',
 'https://www.realtor.com/realestateandhomes-search/California/pg-7',
 'https://www.realtor.com/realestateandhomes-search/California/pg-8',
 'https://www.realtor.com/realestateandhomes-search/California/pg-9',
 'https://www.realtor.com/realestateandhomes-search/California/pg-10',
 'https://www.realtor.com/realestateandhomes-search/California/pg-11',
 'https://www.realtor.com/realestateandhomes-search/California/pg-12',
 'https://www.realtor.com/realestateandhomes-search/California/pg-13',
 'https://www.realtor.com/realestateandhomes-search/California/pg-14',
 'https://www.realto

## Working Code:

In [6]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
for url in url_list:
    #URL and browser
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    list_property=soup.find_all('li', class_='jsx-2423110403 component_property-card')

    #extracting all the information
    for i in range(0,len(list_property)):
        property_type=list_property[i].find('span', class_='jsx-2810661766')
        #print(property_type.text)
        price=list_property[i].find('div',class_="jsx-2810661766 price srp-page-price")
        prop_details=list_property[i].find('ul',class_='jsx-946479843 property-meta list-unstyled property-meta-srpPage')
        address=list_property[i].find('div',class_='jsx-2810661766 address ellipsis srp-page-address')
        link=list_property[i].find('div',class_="jsx-2810661766 detail-wrap has-cta fixed-wrapper")

        prop_details_dict={"Bed":None,
                            "Bath":None,
                            "House Square Feet":None,
                            "Lot":None}
        label_list=prop_details.find_all("span",class_="jsx-946479843 meta-label")
        value_list=prop_details.find_all("span",class_="jsx-946479843 meta-value")
        if(len(label_list)==len(value_list)):
            for l in range(0,len(label_list)):
                if(label_list[l].text=="bed"):
                    prop_details_dict["Bed"]=value_list[l].text
                elif(label_list[l].text=="bath"):
                    prop_details_dict["Bath"]=value_list[l].text
                elif(label_list[l].text=="sqft"):
                    prop_details_dict["House Square Feet"]=f'{value_list[l].text} {label_list[l].text}'
                elif(label_list[l].text=="sqft lot" or label_list[l].text=="acre lot" ):
                    prop_details_dict["Lot"]=f'{value_list[l].text} {label_list[l].text}'
        if (property_type.text and price.span.text and link.a["href"] and address.text):
            #print(prop_details_dict)
            #converting price to integer
            price=int(''.join(n for n in price.span.text if n.isdigit()))
            #print(price)
            #print(address.text)
            #finding Zipcode from address
            zip_code=''
            for j in range(1,6):
                zip_code=zip_code+address.text[-j]
            zip_code=zip_code[::-1]
            #print(zip_code)
            #link
            link=f'https://www.realtor.com/{link.a["href"]}'
            #print(link)
            #dictionary to post in mongo DB
            post={
                "property_type":property_type.text,
                "Price":price,
                "No of beds":prop_details_dict['Bed'],
                "No of bath":prop_details_dict['Bath'],
                "Sqaure Feet":prop_details_dict['House Square Feet'],
                "Lot":prop_details_dict['Lot'],
                "Reference Link":link,
                "Address":address.text,
                "Zip-Code":zip_code
            }

            collection.insert_one(post)
            print("insert_success")
    print("One page Complete")
    time.sleep(random.randint(100,130))
    browser.quit   
    

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\vines\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
One page Complete
One page Complete
One page Complete
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
i

insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
One page Complete
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_

insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
One page Complete
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_

...

insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
One page Complete
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_

insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
One page Complete
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_

insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
One page Complete
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_

insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
One page Complete
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_success
insert_

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url='https://www.realtor.com/realestateandhomes-search/California'
browser.visit(url)
#extracting all the information
for i in range(0,len(list_property)):
    property_type=list_property[i].find('span', class_='jsx-2810661766')
    print(property_type.text)
    price=list_property[i].find('div',class_="jsx-2810661766 price srp-page-price")
    prop_details=list_property[i].find('ul',class_='jsx-946479843 property-meta list-unstyled property-meta-srpPage')
    address=list_property[i].find('div',class_='jsx-2810661766 address ellipsis srp-page-address')
    link=list_property[i].find('div',class_="jsx-2810661766 detail-wrap has-cta fixed-wrapper")

    prop_details_dict={"Bed":None,
                      "Bath":None,
                      "House Square Feet":None,
                      "Lot Square Feet":None}
    label_list=prop_details.find_all("span",class_="jsx-946479843 meta-label")
    value_list=prop_details.find_all("span",class_="jsx-946479843 meta-value")
    for l in range(0,len(label_list)):
        if(label_list[l].text=="bed"):
            prop_details_dict["Bed"]=value_list[l].text
        elif(label_list[l].text=="bath"):
            prop_details_dict["Bath"]=value_list[l].text
        elif(label_list[l].text=="sqft"):
            prop_details_dict["House Square Feet"]=int(''.join(n for n in value_list[l].text if n.isdigit()))
        elif(label_list[l].text=="sqft lot" or label_list[l].text=="acre lot" ):
            prop_details_dict["Lot Square Feet"]=int(''.join(n for n in value_list[l].text if n.isdigit()))
        else:
            prop_details_dict[label_list[l].text]=value_list[l].text
    print(prop_details_dict)
    #converting price to integer
    price=int(''.join(n for n in price.span.text if n.isdigit()))
    print(price)
    print(address.text)
    #finding Zipcode from address
    zip_code=''
    for j in range(1,6):
        zip_code=zip_code+address.text[-j]
    zip_code=zip_code[::-1]
    print(zip_code)
    #link
    link=f'https://www.realtor.com/{link.a["href"]}'
    print(link)
    #dictionary to post in mongo DB
    post={
        "property_type":property_type.text,
        "Price":price,
        "No of beds":prop_details_dict['Bed'],
        "No of bath":prop_details_dict['Bath'],
        "Sqaure Feet":prop_details_dict['House Square Feet'],
        "Lot Square Feet":prop_details_dict['Lot Square Feet'],
        "Reference Link":link,
        "Address":address.text,
        "Zip-Code":zip_code
    }

    collection.insert_one(post)
    print("insert_success")
    

1
4
3
3
2
3
4
3
4
